## IMPORTING LIBRARIES AND DATA

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  #for label encoding
from sklearn.metrics.cluster import homogeneity_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from numpy.linalg import norm


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from tensorflow.keras.layers import LSTM, MaxPool1D, Dropout, Dense, GlobalMaxPooling1D, Embedding, Activation
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

#from sklearn.externals import joblib
#from google.colab import files



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/somyalalwani9/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/somyalalwani9/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/somyalalwani9/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ImportError: cannot import name 'joblib' from 'sklearn.externals' (/home/somyalalwani9/.local/lib/python3.8/site-packages/sklearn/externals/__init__.py)



## OPENING TRAIN.CSV & VALID.CSV


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

def open_filter_data(filename):
    
    #data = pd.read_csv("drive/MyDrive/SMAI_PROJECT_CLICKBAIT/"+filename+".csv", na_values='?', header=None,) 
    data = pd.read_csv("clickbait-news-detection/"+filename+".csv", na_values='?', header=None,) 
    data.head()
    #print("data.shape before:",end=" ")
    #print(data.shape)
    
    df=data.dropna(axis = 0, how ='any')
    #print("data.shape after:",end=" ")
    #print(df.shape)
    df=df.reset_index(drop=True)
    return df
    

df_train=open_filter_data("train")
#df_train.describe

#print("******")

df_valid=open_filter_data("valid")
#df_valid.describe

df_test=open_filter_data("test")

## PREPROCESSING OF TRAINING DATA & VALIDATION DATA

In [ ]:
def extract_data(df):
    a=df.iloc[1:,2]
    b=df.iloc[1:,1]
    df.iloc[:,3] = df.iloc[:,3].str.replace('news','1')
    df.iloc[:,3] = df.iloc[:,3].str.replace('clickbait','0')
    y_actual=df.iloc[1:,3]
    return a,b,y_actual

def extract_dd(df):
    a=df.iloc[1:,2]
    b=df.iloc[1:,1]
    return a,b
    

a_train,b_train,y_actual=extract_data(df_train)
a_valid,b_valid,y_valid=extract_data(df_valid)
a_test,b_test=extract_dd(df_test)

# print(a)
# print("**********")
# print(b)
# print("**********")
# print(y_actual)

# print("#################")

# print(a_valid)
# print("**********")
# print(b_valid)
# print("**********")
# print(y_valid)



1. Converting all sentences to lower case 
2. Writing Abbreviations in full form
3. Removing punctuations (Normalisation)
4. Removing Stop words (Normalisation)
5. Lemmatizing the data


In [ ]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
"'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
"didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
"hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
"he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
"i'd": "I would", "i'd've": "I would have", "i'll": "I will", "i'll've": "I will have",
"i'm": "I am", "i've": "I have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is",
"let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
"mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock",
"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
"shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
"shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that had",
"that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have",
"there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
"we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
"what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is",
"what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
"where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why has", "why've": "why have",
"will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
"wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
"y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
"you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"
}


def pre_process_data(a,b):
    final_lines=[]
    final_headings=[]


    for line in range(1,int(0.25*len(a))):
      #print(a[line])
      #print(type(a[line]))
      #print("*********")
      a[line]=(a[line]).lower()     #Converting all sentences to lower case 
      for word in a[line].split():
        if word in contractions:
            a[line]=a[line].replace(word, contractions[word.lower()])  #Writing Abbreviations in full form
      tokens = word_tokenize(a[line].lower()) 
      words = [word for word in tokens if word.isalpha()]    #Removing punctuations
      final_word = [w for w in words if not w in stop_words]     #Removing Stop words 
      final_words = [lemmatizer.lemmatize(w) for w in final_word]     #Lemmatizing words
      ans=""
      for x in final_words:
        ans= ans+ " "+x
      final_lines.append(ans.lstrip())
      b[line]=(b[line]).lower()     #Converting all sentences to lower case 
      for word in b[line].split():
        if word in contractions:
            b[line]=b[line].replace(word, contractions[word.lower()])  #Writing Abbreviations in full form
      tokens = word_tokenize(b[line].lower()) 
      words = [word for word in tokens if word.isalpha()]    #Removing punctuations
      final_head = [w for w in words if not w in stop_words]     #Removing Stop words 
      final_heads = [lemmatizer.lemmatize(w) for w in final_head]     #Lemmatizing words
      ans=""
      for x in final_heads:
        ans= ans+ " "+x
      final_headings.append(ans.lstrip())
    return final_headings,final_lines


def vectorisation(final_lines):    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(final_lines)
    final_word_vector=[]
    i=0
    while (i<X.shape[0] and i+10000<X.shape[0]):
        final_word_vector.extend(X[i:i+10000].toarray())
        i+=10000

    final_word_vector.extend(X[i:].toarray())
    return final_word_vector


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))



# pre-process all data

In [ ]:
final_headings,final_lines = pre_process_data(a_train,b_train)
final_headings_valid,final_body_valid = pre_process_data(a_valid,b_valid)
final_headings_test,final_body_test = pre_process_data(a_test,b_test)

In [ ]:
# print(len(final_headings),
#       len(final_lines),
#       len(final_headings_valid),
#       len(final_body_valid),
#       len(final_headings_test),
#       len(final_body_test)
# )

In [ ]:
final_headings.extend(final_lines)
final_headings.extend(final_headings_valid)
final_headings.extend(final_body_valid)
final_headings.extend(final_headings_test)
final_headings.extend(final_body_test)

In [ ]:
del stop_words
del lemmatizer
del final_lines



In [ ]:
print(len(final_headings))

In [ ]:
x=vectorisation(final_headings)

In [ ]:
#print(len(x))

In [ ]:
print(type(x))

In [ ]:
final_heading_vector=np.array(x[:4968])
joblib.dump(final_heading_vector, 'final_heading_vector.pkl')
#del final_heading_vector


In [ ]:
#print(final_heading_vector.shape)

In [ ]:
a=9936

final_body_vector=np.array(x[4968:a])
#joblib.dump(final_body_vector,  'final_body_vector.pkl')   
# del final_body_vector


In [ ]:
#print(final_body_vector.shape)

In [ ]:
validation_heading_vector=np.array(x[a:a+705])
#joblib.dump(validation_heading_vector,  'validation_heading_vector.pkl')
# del validation_heading_vector


In [ ]:
#print(validation_heading_vector.shape)

In [ ]:
validation_body_vector=np.array(x[a+705:a+1410])
#joblib.dump(validation_body_vector,  'validation_body_vector.pkl')
# del validation_body_vector


In [ ]:
test_heading_vector=np.array(x[a+1410:a+2816])
#joblib.dump(test_heading_vector,  'test_heading_vector.pkl')
# del test_heading_vector


In [ ]:
#print(test_heading_vector.shape)

In [ ]:
test_body_vector=np.array(x[a+2816:])
#joblib.dump(test_body_vector, 'test_body_vector.pkl')
# del test_body_vector

In [ ]:
#print(test_body_vector.shape)

In [ ]:
# print(validation_heading_vector.shape)
# print(validation_body_vector.shape)
# print(y_valid.shape)


# for testing data

In [ ]:
# print(len(final_heading_vector))
# print(len(final_body_vector))
# print(len(validation_heading_vector))
# print(len(validation_body_vector))
# print(len(test_heading_vector))
# print(len(test_body_vector))



In [ ]:
y_actual=y_actual[:4968].astype(float)
y_valid=y_valid[:705].astype(float)


In [ ]:
y_actual=np.array(y_actual)
y_valid=np.array(y_valid)

In [ ]:
# joblib.dump(y_actual,  'y_actual.pkl')
# #del y_actual

# joblib.dump(y_valid,  'y_valid.pkl')
# #del y_valid

## MODEL

In [ ]:
# pickle.load('final_heading_vector.pkl')
final_body_vector=joblib.load('final_body_vector.pkl')
validation_heading_vector=joblib.load('validation_heading_vector.pkl')
validation_body_vector=joblib.load('validation_body_vector.pkl')
test_heading_vector=joblib.load('test_heading_vector.pkl')
test_body_vector=joblib.load('test_body_vector.pkl')
y_actual=joblib.load('y_actual.pkl')
y_valid=joblib.load('y_valid.pkl')

In [ ]:

def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Dense(128, activation='relu')(input)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    # x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    # x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    return Model(input, x)


def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)



In [ ]:
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:
# print(validation_heading_vector.shape)
# print(validation_body_vector.shape)
# print(y_valid.shape)

In [ ]:
# network definition
input_shape = final_heading_vector.shape[1:]
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# train
rms = RMSprop()
#rms = Adam()
#rms = SGD()

model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
# train
history = model.fit([final_heading_vector, final_body_vector], y_actual,
          batch_size=128,
          epochs=10,
          validation_data=([validation_heading_vector,validation_body_vector], y_valid))

y_pred = model.predict([test_heading_vector, test_heading_vector])
#tr_acc = compute_accuracy(y_test, y_pred_tr)
#te_acc = compute_accuracy(y_test, y_pred_tr)

#print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
#print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))
